In [1]:
import mxnet as mx
from mxnet import nd
import warnings
warnings.filterwarnings('ignore')

### 1D Fast Global Smoother
对于1D的情况，可以定义一个WLS能量函数如下
$$
J(u) = \sum_{x} ((u_x^h - f_x^h)^2 + \lambda_t \sum_{i \in N_h(x)} w_{x, i}(g^h)(u_x^h - u_i^h)^2)
$$
需要极小化这个能量函数，相当于$\Delta J(u) = 0$，可以等价于求解下面的线性系统:
$$
(I_h + \lambda_t A_h) u_h = f_h
$$
**使用下面的迭代方法**

首先计算向前方向
$$
\tilde{c_x} = c_x / (b_x - \tilde{c_{x-1}} a_x) \\
\tilde{f^h_x} = (f_x^h - \tilde{f_{x-1}^h} a_x) / (b_x - \tilde{c_{x-1}} a_x) \\
(x = 1, \cdots, W-1)
$$

然后计算向后方向
$$
u_x^h = \tilde{f^h_x} - \tilde{c_x} u_{x+1}^h \quad(x = W - 2, \cdots, 0) \\
u_{W-1}^h = \tilde{f_{W-1}^h}
$$

In [2]:
def cw_1d(p, q, g, sigma):
    '''
    计算1d上曲线g上不同位置p和q的相似性，使用sigma作为一个范围度量
    g: ndarray，W
    p: int
    q: int
    sigma: float
    '''
    norm = nd.norm(g[p] - g[q])
    return nd.exp(-norm/sigma)

In [3]:
def compute_lamb(t, T, lamb_base):
    return 1.5 * 4**(T-t) / (4 ** T - 1) * lamb_base

In [10]:
def compute_1d_fast_global_smoother(lamb, f, g):
    '''
    求解1d fast global smoother，给出lambda, f和g，进行前向和反向的递归求解
    '''
    w = f.shape[0]
    _c = nd.zeros(shape=w)
    _c[0] = -lamb * cw_1d(0, 1, g) / (1 + lamb * cw_1d(0, 1, g))
    _f = nd.zeros(shape=w)
    _f[0] = f[0] / (1 + lamb * cw_1d(0, 1, g))
    # 递归前向计算
    for i in range(1, w):
        _c[i] = -lamb * cw_1d(i, i + 1, g) / (
            1 + lamb * (cw_1d(i, i - 1) + cw_1d(i, i + 1)) +
            lamb * _c[i - 1] * cw_1d(i, i - 1, g))
        _f[i] = (f[i] + _f[i - 1] * lamb * cw_1d(i, i - 1, g)) / (
            1 + lamb * (cw_1d(i, i - 1, g) + cw_1d(i, i + 1, g)) +
            lamb * _c[i - 1] * cw_1d(i, i - 1))
    u = nd.zeros(shape=w)
    u[w - 1] = _f[w - 1]
    # 递归向后计算
    for i in range(w - 2, -1, -1):
        u[i] = _f[i] - _c[i] * u[i + 1]
    return u

对于一张2D的图片，分别在水平和竖直上应用1D的solver就能够求解，但是会出现streaking artifact的问题，为了解决这个问题，在每一次的迭代中，可以修改$\lambda$的值，因为这个值在每次迭代中对稀疏光滑具有显著的减少。

In [19]:
def Separable_global_smoother(f, g, T, lamb_base):
    '''
    全局图片光滑算法，输入是f和g，f是2D image，g是2D guide image
    '''
    H, W = f.shape
    u = nd.zeros(shape=(H, W))
    for t in range(1, T+1):
        # 计算每一步迭代的lambda_t
        lamb_t = compute_lamb(t, T, lamb_base)
        # horizontal
        for y in range(0, H):
            f_h = f[:, y]
            u[:, y] = compute_1d_fast_global_smoother(lamb_t, f_h, f_h)
        # vertical
        for x in range(0, W):
            f_v = f[x, :]
            u[x, :] = compute_1d_fast_global_smoother(lamb_t, f_v, f_v)
    return u